In [85]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [86]:
X = np.load('combined.npy')
y = np.load('segmented.npy')

In [87]:
print(X.shape, y.shape)

(2000, 64, 84) (2000, 64, 84, 10)


In [88]:
#Shuffle
X,y = shuffle(X,y)

X = X.astype(np.float32)
y = y.astype(np.float32)

In [89]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [90]:
X_train = (X_train.astype(np.float32)-127)/127.0
X_test = (X_test.astype(np.float32)-127)/127.0

In [91]:
from tensorflow.core.framework import graph_pb2

In [92]:
graph_def = graph_pb2.GraphDef()

In [93]:
def load_graph(graph_path):
    with open(graph_path, "rb") as f:
        graph_def.ParseFromString(f.read())
        with tf.Graph().as_default() as graph:
            input_big = tf.placeholder(dtype=tf.float32, shape=(None, 64,84,1), name='input_big_image')
            tf.import_graph_def(graph_def, name="", input_map={"reshaped_image": input_big})
            return graph

In [94]:
nb_classes=10

In [95]:
def nn(nb_classes):
    encoder = load_graph('./checkpoints/frozen_graph.pb')
    with encoder.as_default():
        labels = tf.placeholder(dtype=tf.int32, shape=(None, y.shape[1], y.shape[2], y.shape[3]), name='segmentation_labels')
        learning_rate_ph = tf.placeholder(dtype=tf.float32, name='learning_rate')
        input_ph = encoder.get_tensor_by_name('input_big_image:0')
        maxpool1_out = encoder.get_tensor_by_name('pool1/MaxPool:0')
        maxpool2_out = encoder.get_tensor_by_name('poool2/MaxPool:0')
    
        conv_11 = tf.layers.conv2d(maxpool2_out,32,1,1,activation=tf.nn.relu, name='conv11')
        
        #decoder starts here
        
        conv_11_2 = tf.layers.conv2d(maxpool2_out,10,1,1,activation=tf.nn.relu, name='conv112')
        
        trans_conv = tf.layers.conv2d_transpose(conv_11_2, 10, 4, 2,activation=tf.nn.relu, name='trans_conv1')
        
        added_skip = tf.add(trans_conv, tf.layers.conv2d(maxpool1_out,10,1,1,activation=tf.nn.relu))
        
        trans_conv_2 = tf.layers.conv2d_transpose(added_skip, 10, 6, 2,activation=tf.nn.relu, name='trans_conv2')
        
        out =tf.reshape(trans_conv_2,[-1,nb_classes], name='logits')
        
        labels_flat = tf.reshape(labels,[-1,nb_classes])
        
        # Define the cost
        cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=out, labels=tf.one_hot(labels_flat,depth=10)), name='cost')
        
        # Apply an optimizer
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate_ph).minimize(cost)

        return labels, learning_rate_ph, input_ph, cost, optimizer

In [96]:
labels, learning_rate_ph, input_ph, cost, optimizer = nn(nb_classes)


In [97]:
def get_next_batch(batch_size, X, y):
    for i in range(0,len(y),batch_size):
        start = i
        end = min(len(y), i+batch_size)
        yield X[start:end, :] , y[start:end]

In [98]:
def eval_cost(cost_op, X, y, batch_sz):
    total_cost = 0
    nb_batches = 0
    for batch_X,batch_y in get_next_batch(batch_size,X,y):
        feed_dict={img_placeholder: batch_X, labels: batch_y}
        total_cost += cost.eval(feed_dict=feed_dict)
        nb_batches += 1
    return total_cost / nb_batches

def eval_accuracy(output_op, X, y, batch_sz):
    nb_batches = 0
    total_acc = 0
    for batch_X, batch_y in get_next_batch(batch_sz,X,y):
        feed_dict={img_placeholder: batch_X, labels: batch_y}
        y_predicted = np.argmax(output.eval(feed_dict=feed_dict),1)
        total_acc += accuracy_score(batch_y,y_predicted)
        nb_batches += 1
    return 100*total_acc/nb_batches

In [99]:
from sklearn.metrics import accuracy_score
from tqdm import tqdm

epochs = 75
batch_size = 512
learning_rate = 0.001
batches_per_epoch = int(X_train.shape[0]/batch_size)
n_train = int(len(y_train)*0.8)



# Launch the graph
with tf.Session() as sess:
    summary_writer = tf.summary.FileWriter('./',sess.graph)
    
    sess.run(tf.global_variables_initializer())
    
    for epoch in tqdm(range(epochs)):
        # Shuffle data for cross validation
        indices = np.array(range(len(y_train)))
        np.random.shuffle(indices)
        
        train_indices = indices[: n_train]
        val_indices = indices[n_train:]
        

        
        y_train_fold, X_train_fold, y_val_fold, X_val_fold = y_train[train_indices], X_train[train_indices, :], \
        y_train[val_indices], X_train[val_indices, :]
        
        # Loop over all training batches
        for x, y in get_next_batch(batch_size, X_train_fold, y_train_fold):
            sess.run(optimizer, 
                     feed_dict={input_ph:x, labels:y, learning_rate_ph:learning_rate})
        
        if (epoch+1) % 5 == 0:
            training_cost = eval_cost(cost, X_train_fold, y_train_fold, batch_size)
            training_acc  = eval_accuracy(output, X_train_fold, y_train_fold, batch_size)
            valid_acc  = eval_accuracy(output, X_val_fold, y_val_fold, batch_size)
            test_acc  = eval_accuracy(output, X_test, y_test, batch_size)
            #print(training_cost, training_acc, valid_acc, test_acc)
            
            summary = tf.Summary()
            summary.value.add(tag="Train Cost", simple_value=training_cost)
            summary.value.add(tag="Train Accuracy", simple_value=training_acc)
            summary.value.add(tag="Validation Accuracy", simple_value=valid_acc)
            summary.value.add(tag="Test Accuracy", simple_value=test_acc)
            summary_writer.add_summary(summary,epoch+1)
            summary_writer.flush()

    
    print("Optimization Finished!")
    summary_writer.close()
    # Save the graph structure only.
    # Need to save weights in a checkpoint file separately.
    # The graph structure is saved twice, once as a text file 
    # with extension '.pbtxt' and then as a binary file with 
    # extension '.pb'. This is done for demonstration only.
    tf.train.write_graph(
    tf.get_default_graph().as_graph_def(),
    './checkpoints/',
    'mnist.pbtxt',
    as_text=True)
    
    tf.train.write_graph(
    tf.get_default_graph().as_graph_def(),
    './checkpoints/',
    'mnist.pb',
    as_text=False)
    
    saver = tf.train.Saver()
    saver.save(sess,'./checkpoints/mnist.ckpt',epoch+1)

  0%|                                                                                           | 0/75 [00:00<?, ?it/s]


TypeError: Cannot interpret feed_dict key as Tensor: Tensor Tensor("input_big_image:0", shape=(?, 64, 84, 1), dtype=float32) is not an element of this graph.